 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [1]:
import pandas as pd
import sys
sys.path.append('..')
from lib import Node
from lib import Tree

In [2]:
df = pd.read_pickle('../data/parsed/pickled_data_test.pickle')

In [3]:
rows = [1, 30, 32, 43, 46]

In [4]:
members = df.loc[rows]['Label'].values

In [5]:
members

array(['-1', '-1', '-1', '-1', '-1'], dtype=object)

In [6]:
members == '-1'

array([ True,  True,  True,  True,  True], dtype=bool)

In [7]:
a = members[members == '-1']
a

array(['-1', '-1', '-1', '-1', '-1'], dtype=object)

TODO:

- Dates to some kind of number
- from, to: split using membership
- dates: split on number
- subject: find subjects that are equal, or lsa matrix


In [8]:
one = df.loc[df['Scenario'] == '401']
one_labeled = one.loc[one['Label'] != '-1']
one_labeled.shape

(710, 21)

In [9]:
def get_first(l):
    return l[0]

In [10]:
df['From'].apply(lambda x: x[0]).unique()[0]

'david.allan@enron.com'

In [11]:
##test.loc[test['From'].apply(lambda x: x[0]) == 'david.forster@enron.com'].index.valuestest[0:1]

In [12]:
test = df.loc[df['Scenario'] == '401']
test_labeled = test.loc[test['Label'] != '-1']
test = test_labeled[0:100]

In [13]:
test.shape

(100, 21)

In [14]:
test = test.sample(frac=1)
test = test.reset_index(drop=True)

In [15]:
ls = ['From']

In [16]:
tree = Tree(test, 4, None, range(test.shape[0]), ls, ls)

In [17]:
tree.fit()

The node is pure, no more splitting
L From david.forster@enron.com [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77
 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99]
The node has no document feed, no more splitting
L From david.forster@enron.com [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77
 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99]
The node has no document feed, no more splitting
L From david.forster@enron.com [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 6

In [18]:
tree.visualize()

ZeroDivisionError: division by zero

In [ ]:
%%bash
cd vis
dot -Tpng tree.dot -o tree.png
dot -Tsvg tree.dot -o tree.svg

In [ ]:
from IPython.display import Image
Image(filename='vis/tree.png')

In [ ]:
test.loc[test['From'].apply(lambda x: x[0]) == 'david.forster@enron.com'].index.values

In [ ]:
members = test.loc[range(0,100)]
left_members = members.loc[members['From'] == 'david.forster@enron.com'].index.values
members.loc[members['From'].apply(lambda x: x [0] == 'david.forster@enron.com')]